# Generate Hot Off Gas Background (511-keV gammas)

Copied from https://github.com/yuntsebaryon/SimpleCosmics/blob/main/HOG/genHOG.ipynb, but change to create hepevt files

TODO: Check with Rex about the (x, y, z, t) for the HOG gammas and rate; 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
outdir = '/Users/yuntse/data/coherent/SNeNDSens/gen/HOG'

if os.path.exists(outdir):
    raise FileExistsError(f"Directory '{outdir}' already exists!")
else:
    os.makedirs(outdir)

## Generate the location and time: xyzt

In [ ]:
# location and time in [cm, ns]
def assignXYZT(xlim, ylim, zlim, tlow, thigh, nSamples):
    xyzt = np.array([ [ x, y, z, t ] 
                      for x, y, z, t in zip(np.random.uniform(-xlim, xlim, nSamples), 
                                            np.random.uniform(-ylim, ylim, nSamples),
                                            np.random.uniform(-zlim, zlim, nSamples),
                                            np.random.uniform(tlow, thigh, nSamples) ])
    return xyzt

In [2]:
gPdg = 22
# unit: GeV
gMass = 0
gEnergy = 511e-6

In [ ]:
xlim =
ylim =
zlim =
tlow =
thigh =

In [3]:
nEvents = 50000
nGammaPerEvent = 73
nParticlePerVtx = 1
nEventPerFile = 5000
nFiles = int(nEvents/nEventPerFile)
nFiles

10

In [5]:
for iFile in range(nFiles):
    outFile = f'{outdir}/HOG_{iFile:04d}.hepevt'

    with open( outFile, 'w') as f:
        for iEvtPerFile in range(nEventPerFile):
            iEvt = iFile*nEventPerFile + iEvtPerFile

            # assign xyzt
            xyzt = assignXYZT(xlim, ylim, zlim, tlow, thigh, nGammaPerEvent)
            
            for iGamma in range( nGammaPerEvent ):
                f.write( f'{iEvt} {iGamma} {nParticlePerVtx}\n')

                # ISTHEP IDHEP JMOHEP1 JMOHEP2 JDAHEP1 JDAHEP2 PHEP1 PHEP2 PHEP3 PHEP4 PHEP5 VHEP1 VHEP2 VHEP3 VHEP4
                # final-state particle
                ISTHEP = 1
                IDHEP = gPdg
                # The JMOHEP1, JMOHEP2, JDAHEP1, and JDAHEP2 entries record the indices (between 1 and NHEP, inclusive) 
                # of particles in the event record that correspond to the first mother, second mother, first daughter, 
                # and last daughter of the current particle, respectively. 
                JMOHEP1 = 0
                JMOHEP2 = 0
                JDAHEP1 = 0
                JDAHEP2 = 0
                
                # Calculate the 4-momentum
                E = gEnergy
                p = gEnergy
                px = 0.
                py = -gEnergy
                pz = 0.

                # Fill momentum
            
                PHEP1 = px
                PHEP2 = py
                PHEP3 = pz
                PHEP4 = E
                PHEP5 = gMass
                VHEP1 = xyzt[iGamma][0]
                VHEP2 = xyzt[iGamma][1]
                VHEP3 = xyzt[iGamma][2]
                VHEP4 = xyzt[iGamma][3]
                f.write( f'{ISTHEP} {IDHEP} {JMOHEP1} {JMOHEP2} {JDAHEP1} {JDAHEP2} {PHEP1} {PHEP2} {PHEP3} {PHEP4} {PHEP5} {VHEP1} {VHEP2} {VHEP3} {VHEP4}\n')
            

saving event 4999 in file 0....
saving event 9999 in file 1....
saving event 14999 in file 2....
saving event 19999 in file 3....
saving event 24999 in file 4....
saving event 29999 in file 5....
saving event 34999 in file 6....
saving event 39999 in file 7....
saving event 44999 in file 8....
saving event 49999 in file 9....
